# **Intercomparison Demo**

**Author:** Adam Yang (ayang115@umd.edu)

**Description** Demo for Intercomparison Script: extracts test points, provided from hand labeled maps, from industry covermaps (Harvest, Copernicus, ESA, GLAD) and determines accuracy across maps.


In [1]:
# !earthengine authenticate

In [1]:
import ee
ee.Initialize()

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ee
from shapely import wkt

import sys
sys.path.append("..")
from src.compare_covermaps import *

c:\Users\adamf\anaconda3\envs\intercomparison\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
set1_maps = [TARGETS["harvest_togo"], TARGETS["harvest_kenya"], TARGETS["harvest_tanzania"], TARGETS["copernicus"], TARGETS["esa"], TARGETS["glad"]]
set1_countries = ["Togo", "Kenya", "Tanzania"]

In [8]:
test1 = TestCovermaps(test_countries=set1_countries, covermaps=set1_maps)

test1_test = pd.read_csv("../../intercomparison-data/extracted/test/tgo_tan_ken_2019_(newkenya).csv").drop("Unnamed: 0", axis=1)
test1_test["geometry"] = test1_test["geometry"].apply(wkt.loads)
test1_test = gpd.GeoDataFrame(test1_test, crs="epsg:4326")

# test1_test = test1.get_test_points()


In [6]:
extracted = test1.extract_covermaps(test1_test)

[Togo] sampling harvest_togo...
[Togo] sampling copernicus...
[Togo] sampling esa...
[Togo] sampling glad...
[Kenya] sampling harvest_kenya...
Extracting error: length of sampled dataset is not the same as testing dataset
[Kenya] sampling copernicus...
[Kenya] sampling esa...
[Kenya] sampling glad...
[Tanzania] sampling harvest_tanzania...
[Tanzania] sampling copernicus...
[Tanzania] sampling esa...
[Tanzania] sampling glad...


In [5]:
a=test1.evaluate()
test1.results["Tanzania"]

evaluating maps...
dataset: harvest_togo | country: Togo
dataset: copernicus | country: Togo
dataset: esa | country: Togo
dataset: glad | country: Togo
dataset: harvest_kenya | country: Kenya
dataset: copernicus | country: Kenya
dataset: esa | country: Kenya
dataset: glad | country: Kenya
dataset: harvest_tanzania | country: Tanzania
dataset: copernicus | country: Tanzania
dataset: esa | country: Tanzania
dataset: glad | country: Tanzania


,dataset,country,crop_f1,accuracy,crop_support,noncrop_support,crop_recall,noncrop_recall,crop_precision,noncrop_precision
0,harvest_tanzania,Tanzania,0.708861,0.760573,767,770,0.584094,0.936364,0.901408,0.693269
0,copernicus,Tanzania,0.742681,0.782694,767,770,0.628422,0.936364,0.907721,0.716700
0,esa,Tanzania,0.741627,0.789200,767,770,0.606258,0.971429,0.954825,0.712381
0,glad,Tanzania,0.718494,0.776187,767,770,0.572360,0.979221,0.964835,0.696858


In [6]:
test1.results["Togo"]

,dataset,country,crop_f1,accuracy,crop_support,noncrop_support,crop_recall,noncrop_recall,crop_precision,noncrop_precision
0,harvest_togo,Togo,0.747541,0.781250,152,200,0.750000,0.805,0.745098,0.809045
0,copernicus,Togo,0.569170,0.690341,152,200,0.473684,0.855,0.712871,0.681275
0,esa,Togo,0.629787,0.752841,152,200,0.486842,0.955,0.891566,0.710037
0,glad,Togo,0.562771,0.713068,152,200,0.427632,0.930,0.822785,0.681319


In [7]:
test1.results["Kenya"]

,dataset,country,crop_f1,accuracy,crop_support,noncrop_support,crop_recall,noncrop_recall,crop_precision,noncrop_precision
0,harvest_kenya,Kenya,0.507463,0.883803,36,248,0.472222,0.943548,0.548387,0.924901
0,copernicus,Kenya,0.567568,0.888889,36,252,0.583333,0.932540,0.552632,0.940000
0,esa,Kenya,0.391304,0.902778,36,252,0.250000,0.996032,0.900000,0.902878
0,glad,Kenya,0.757576,0.944444,36,252,0.694444,0.980159,0.833333,0.957364


### **Testing**

In [11]:
ic = ee.ImageCollection(ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filter(ee.Filter.date("2019-01-01", "2020-01-01")).select(["crops"]).mode())
fc = ee.FeatureCollection(
            test1_test.apply(lambda x: create_point(x), axis=1).to_list()
        )

extract_points(ic, fc)

,geometry,binary,lat,lon,mode
0,POINT (1.17247 9.87591),0,9.875907,1.172471,0.032129
1,POINT (1.37469 9.18011),0,9.180105,1.374695,0.027983
2,POINT (1.19639 9.27531),0,9.275314,1.196385,0.029688
3,POINT (0.70517 7.34658),0,7.346575,0.705167,0.036812
4,POINT (0.62982 6.77734),0,6.777337,0.629817,0.028121
...,...,...,...,...,...
2172,POINT (35.97636 -2.99837),1,-2.998368,35.976357,0.141485
2173,POINT (35.98211 -2.99638),1,-2.996376,35.982109,0.148479
2174,POINT (35.97994 -2.99896),1,-2.998963,35.979944,0.164197
2175,POINT (35.98412 -2.99794),1,-2.997940,35.984116,0.078750
